In [1]:
#General stuff
import time
import datetime
import glob
import re
import pandas as pd
from collections import Counter
import progressbar
import jellyfish
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
import statsmodels.formula.api as smf

## Pulling in all the files

In [2]:
df = pd.read_csv('data/df_final_harm.csv')

## I want to look for repetitions "sozialrechtliche Abteilungen"
And I'm working with the years, as the whole long string consisting of 15'000 verdicts is just too long. The Kernel dies regularly. Which really isn't much fun.

In [3]:
df_sozrecht = df[df['recht_harm'] == 'Sozialrecht']
sozrecht = df_sozrecht[df_sozrecht['Language'] == 'De']['Text-Nummer']

In [4]:
df_sozrecht['Datetime'] = pd.to_datetime(df_sozrecht['Datetime'], format="%Y-%m-%d")
df_sozrecht.index = df_sozrecht['Datetime']

/Users/barneyjs/.virtualenvs/master/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Listing all the years I need. 

In [8]:
years = ['2007', '2008', '2009', 
         '2010', '2011', '2012', 
         '2013', '2014', '2015', 
         '2016', '2017']

In [17]:
l = list(df_sozrecht['Text-Nummer'])
n = 1000

In [18]:
txt_lsts = [l[i:i + n] for i in range(0, len(l), n)]

In [20]:
df_ = pd.DataFrame(columns=['index', 0])
bar = progressbar.ProgressBar()

for txt_lst, i in zip(txt_lsts, bar(range(len(txt_lsts)))):

    long_str = ''
    
    for txt in txt_lst:
    
        file = open('txt_files/' + txt, 'r')
        text = file.read()
        #Making sure the line breaks '/n', 'xa0' and double spaces are removed
        text = text.replace('\n', ' ').replace('\xa0', '').replace('  ', ' ')
        #Removing these automated notifications
        text = text.replace('Wichtiger Hinweis:Diese Website wird in älteren Versionen von Netscape ohne graphische Elemente dargestellt. Die Funktionalität der Website ist aber trotzdem gewährleistet. Wenn Sie diese Website regelmässig benutzen, empfehlen wir Ihnen, auf Ihrem Computer einen aktuellen Browser zu installieren.Zurück zur Einstiegsseite Drucken Grössere Schrift', '')
        text = text.replace('Vorwärts ähnliche Leitentscheide suchenähnliche Urteile ab 2000 suchen Drucken nach oben', '')
        text = text.replace('Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal', '')
        text = text.replace('Navigation Neue Suche Zurück zum Suchresultat Rang: Zurück 180', '')
        text = text.replace('Navigation Neue Suche Zurück zum Suchresultat Rang:1 ähnliche Leitentscheide suchenähnliche Urteile ab 2000 suchen Drucken nach oben', '')
    
        long_str = long_str + text
        
    df_new = pd.DataFrame(pd.Series( long_str[ix:ix+500] for ix in list(range(len(long_str))) ).value_counts()).reset_index()
    df_new = df_new[df_new[0]> 1]
    
    frames = [df_, df_new]
    df_ = pd.concat(frames)
    

 95% (21 of 22) |########################  | Elapsed Time: 0:15:53 ETA: 0:00:46

# Saving off the result
Later, I can just jump to this step, I don't need to repeat the steps above

In [21]:
df_.to_csv('500Characters.csv')

In [22]:
df_ = pd.read_csv('500Characters.csv')

In [23]:
del df_['Unnamed: 0']

In [24]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3547242 entries, 0 to 3547241
Data columns (total 2 columns):
index    object
0        float64
dtypes: float64(1), object(1)
memory usage: 54.1+ MB


In [26]:
df_.columns = [['String', 'Count']]

# Deleting Duplicates

In [27]:
df_ = df_.drop_duplicates(subset='String', keep='first')

In [28]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2647768 entries, 0 to 3547241
Data columns (total 2 columns):
String    object
Count     float64
dtypes: float64(1), object(1)
memory usage: 60.6+ MB


In [34]:
df = df_[df_['Count'] >= 10]

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35327 entries, 0 to 3355870
Data columns (total 2 columns):
String    35327 non-null object
Count     35327 non-null float64
dtypes: float64(1), object(1)
memory usage: 828.0+ KB


# Matching similar Strings
Using Jellyfish and remodelling the original df, using it to create a new list of string. Because these ones are the ones I don't need to consider anymore.

In [ ]:
df_new = pd.DataFrame(columns=['String', 'Count', 'Jaro'])

for count in range(1000):
    
    print(count)
    string_list = list(df['String'])
    value_list = []
    bar = progressbar.ProgressBar()

    for elem, i in zip(string_list, bar(range(len(string_list)))):
    
        value = jellyfish.jaro_distance(string_list[0], elem)
        value_list.append(value)
    
    df['Jaro'] = value_list
    df_unique = df[df['Jaro'] >= .825].head(1)

    frames = [df_new, df_unique]
    df_new = pd.concat(frames)

    df = df[df['Jaro'] <= .825]
    del df['Jaro']

  2% (895 of 35327) |                      | Elapsed Time: 0:00:00 ETA: 0:00:05

0


 98% (34880 of 35327) |################### | Elapsed Time: 0:00:05 ETA: 0:00:00/Users/barneyjs/.virtualenvs/master/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
  3% (1335 of 35142) |                     | Elapsed Time: 0:00:00 ETA: 0:00:04

1


  3% (1331 of 35041) |                     | Elapsed Time: 0:00:00 ETA: 0:00:04

...

 26% (942 of 3542) |######                 | Elapsed Time: 0:00:00 ETA: 0:00:00

790


 25% (889 of 3509) |#####                  | Elapsed Time: 0:00:00 ETA: 0:00:00

791


 79% (2799 of 3509) |#################     | Elapsed Time: 0:00:00 ETA: 0:00:00

# Cleaning the New DF and Creating a String

In [ ]:
df_new.info()

In [ ]:
del df_new['Count']
del df_new['Year']
del df_new['Jaro']

In [ ]:
len(list(df_new['String']))

In [ ]:
string_list = list(df_new['String'])

# Iterating through all the files

Functions to pull out relevant information

In [ ]:
def extracting_date(doc):
    Datum = re.findall(r"[0-9]+\.[0-9]+\.20[0-9]+", doc)
    try:
        return Datum[0]
    except:
        pass

In [ ]:
def extracting_akten_nummer(doc):
    an = re.findall(r"[0-9]*[A-Z][.]*[_]*[ ]*[0-9]+/[0-9]+", doc)
    try:
        an = an[0]
        an = an.replace('_', ' ')
        return an
    except:
        pass

In [ ]:
#df = pd.read_csv('data/df_final_harm.csv')
#df_sozrechtI = df[df['recht_harm'] == 'Sozialrecht']
#sozrecht = df_sozrechtI[df_sozrechtI['Language'] == 'De']['Text-Nummer']

In [ ]:
soz_lst = list(df_sozrecht)

In [ ]:
txt_elems = []
bar = progressbar.ProgressBar()

for text_file, i in zip(soz_lst, bar((range(len(soz_lst))))):
    
    file = open('txt_files/' + text_file, 'r')
    text = file.read()
    file.close()
    
    #Special formatting
    text = text.replace('\n', ' ').replace('\xa0', '').replace('  ', '')
    text = text.replace('  ', ' ')
    
    Datum = extracting_date(text)
    Aktennummer = extracting_akten_nummer(text)
    
    if string_list[0] in text:
        boolean = 1
    else:
        boolean = 0
    
    mini_dict = {'Text Nummer': text_file,
                 'Datum': Datum,
                 string_list[0][:20]: boolean,
                 'Aktennummer': Aktennummer}
    
    txt_elems.append(mini_dict)

In [ ]:
df_bools = pd.DataFrame(txt_elems)

# Repeat what I did above, now attaching the lists to df_bools

In [ ]:
bar = progressbar.ProgressBar()

for string, i in zip(string_list, bar(range(len(string_list)))):
    
    bool_list = []
    
    for text_file in soz_lst:
    
        file = open('txt_files/' + text_file, 'r')
        text = file.read()
        file.close()
    
        #Special formatting
        text = text.replace('\n', ' ').replace('\xa0', '').replace('  ', '')
        text = text.replace('  ', ' ')
    
        Datum = extracting_date(text)
        Aktennummer = extracting_akten_nummer(text)
    
        if string in text:
            boolean = 1
        else:
            boolean = 0
        
        bool_list.append(boolean)
    
    df_bools[string[:20]] = bool_list

In [ ]:
df_bools.info()

In [ ]:
#df_bools.to_csv('bools750text_snippets.csv')

In [ ]:
df_bools = pd.read_csv('bools750text_snippets.csv')

In [ ]:
del df_bools['Unnamed: 0']

In [ ]:
df_bools.head()

# Making Total Column

In [ ]:
df_bools['TOTAL'] = df_bools.sum(axis=1, numeric_only=True)

In [ ]:
df_bools['TOTAL'].describe()

# Dates

In [ ]:
df_bools['Datetime'] = pd.to_datetime(df_bools['Datum'], format="%d.%m.%Y")
df_bools.index = df_bools['Datetime']

# Plotting and Linear Regression

In [ ]:
df_counts_pyear = pd.DataFrame(df_bools.resample('A')['TOTAL'].sum()).reset_index()

In [ ]:
def change(elem):
    elem = str(elem)
    elem = int(elem.replace('-12-31 00:00:00', ''))
    return elem

In [ ]:
df_counts_pyear['Years'] = df_counts_pyear['Datetime'].apply(change)

In [ ]:
lm = smf.ols(formula="TOTAL~Years",data=df_counts_pyear).fit()

In [ ]:
lm.params

In [ ]:
intercept, slope = lm.params

In [ ]:
df_counts_pyear.plot(x="Years",y="TOTAL")
plt.plot(df_counts_pyear["Years"],slope*df_counts_pyear["Years"]+intercept,"-",color="red") 